# Using Inbuilt Naive Bayes Classifier

In [1]:
import numpy as np
import pandas as pd
import operator
import os,sys
from sklearn import model_selection
import re,string

In [2]:
stop_word=["a","about","above","after","again","against","all","am","an","and","any","are","as","at","be","because","been","before","being","below","between","both","but",
"by","could","did","do","does","doing","down","during","each","few","for","from","further","had","has","have","having","he","he'd","he'll","he's","her",
"here","here's","hers","herself","him","himself","his","how","how's","i","i'd","i'll","i'm","i've","if","in","into","is","it","it's","its","itself","let's","me",
"more","most","my","myself","nor","of","on","once","only","or","other","ought","our","ours","ourselves","out","over","own","same","she",
"she'd","she'll","she's","should","so","some","such","than","that","that's","the","their","theirs","them","themselves","then","there","there's",
"these","they","they'd","they'll","they're","they've","this","those","through","to","too","under","until","up","very","was","we","we'd",
"we'll","we're","we've","were","what","what's","when","when's","where","where's","which","while","who","who's","whom","why","why's","with",
"would","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves"]

In [4]:
X  =[] 
Y = []
for category in os.listdir("D:\\Text Classification\\Datasets"):
    for document in os.listdir("D:\\Text Classification\\Datasets\\"+category):
        with open("D:\\Text Classification\\Datasets\\"+category+'/'+document, "r") as f:
            X.append((document,f.read()))
            Y.append(category)

In [ ]:
print(type(X))
print(type(X[0]))
print(type(X[0][0]))
print(type(X[0][1]))
print(type(Y))
print(X[0])

In [6]:
x_train,x_test,y_train,y_test=model_selection.train_test_split(X,Y)

In [ ]:
print(len(x_train))
print(len(x_test))

In [ ]:
sample_text="Hey! I am Yatin. I am Good."
print(re.split(r'\W+',sample_text))

In [ ]:
dic={}
for i in range(len(x_train)):
    word=x_train[i][1].lower()
    stripped=re.split(r'\W+',word)
    for s in stripped:
        if not(s.isalpha()) or s in stop_word or len(s)<=2:
            continue
        if s in dic:
            dic[s]+=1
        else:
            dic[s]=1
dic

In [ ]:
sorted_dic = sorted(dic.items(), key=operator.itemgetter(1),reverse=True)
sorted_dic

In [12]:
import matplotlib.pyplot as plt

In [ ]:
features=sorted_dic
answer1=[]
answer2=[]
for i in range(len(features)):
    answer1.append(i)
    answer2.append(features[i][1])
plt.plot(answer1,answer2)
plt.axis([0,8000,1,5000])
plt.grid()
plt.show()   


In [ ]:
answer1=[features[i][0] for i in range(2000)]
answer1

In [15]:
x_train_dataset=np.zeros([len(x_train),len(answer1)],int)
for i in range(len(x_train)):
    words=x_train[i][1].lower()
    word=re.split(r'\W+',words)
    for j in word:
        if j in answer1:
            x_train_dataset[i][answer1.index(j)]+=1

In [16]:
x_test_dataset=np.zeros([len(x_test),len(answer1)],int)
for i in range(len(x_test)):
    words=x_test[i][1].lower()
    word=re.split(r'\W+',words)
    for j in word:
        if j in answer1:
            x_test_dataset[i][answer1.index(j)]+=1

In [ ]:
print(x_train_dataset)
print("--------------------------")
print(x_test_dataset)

In [18]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [ ]:
clf=MultinomialNB()
clf.fit(x_train_dataset,y_train)
y_pred=clf.predict(x_test_dataset)
print("Score on training data:",clf.score(x_train_dataset,y_train))
print("Score on testing data:",clf.score(x_test_dataset,y_test))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))


# Self Implementation of Naive Baye's

In [20]:
def fit(x_train_dataset,y_train):
    count={}
    total_word=0
    y_train=np.array(y_train)
    count["total_doc"]=len(y_train)
    classes=set(y_train)
    for i in classes:
        temp=0
        x_train_with_i=x_train_dataset[y_train==i]
        temp2=x_train_with_i.shape[0]
        count[i]={}
        for feature in answer1:
            l=(x_train_with_i[:,answer1.index(feature)]).sum()
            count[i][feature]=l
            temp+=l
        count[i]["word_in_class"]=temp
        count[i]["length"]=temp2
        
    
    return count

In [21]:
def probability(x_test,dic,classes):
    prob=np.log(dic[classes]["length"])-np.log(dic["total_doc"])
    feature=list(dic[classes].keys())
    for j in range (len(feature)-2):
        xj=x_test[j]
        if xj==0:
            current_prob=0
        else:
            num=dic[classes][feature[j]]+1
            den=dic[classes]["word_in_class"]+len(dic[classes].keys())-2
            current_prob=np.log(num)-np.log(den)
        prob+=current_prob
    return prob
    

In [22]:
def predict_for_single(x_test,dic):
    first_run=True
    classes=dic.keys()
    for i in classes:
        if i=="total_doc":
            continue
        prob=probability(x_test,dic,i)
        if first_run or prob>best_prob:
            best_prob=prob
            first_run=False
            best_class=i
    return best_class

In [23]:
def predict_(x_test,dic):
    y_pred=[]
    for x in x_test:
        y_pred.append(predict_for_single(x,dic))
    return y_pred

In [24]:
def score(y_test,y_pred):
        count = 0
        for i in range(len(y_pred)):
            if y_pred[i] == y_test[i]:
                count+=1
        return count/len(y_pred)

In [25]:
dictionary=fit(x_train_dataset,y_train)
y_pred=predict_(x_test_dataset,dictionary)


In [ ]:
dictionary

In [27]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import pandas as pd

In [ ]:
print("Score on testing_data:",score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
